In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Function(s) to clean up known outliers in 'VALUES' column of opiod data set
#This processing is specifc to current set of known outliers

#Look for characters "+", "-", "<" and take numeric value before, after, after repsectively
#   e.g "30+" -> 30, "2-3.5" -> 3.5, "<2" -> 2


#Primary function for clean up, DataSeries input, check each known outlier
#
def clean_nan (incol) :
    
    if pd.api.types.is_numeric_dtype(incol) :      # if Series already numeric
        return incol                               # No processing to be done, return
    
    dfs = pd.to_numeric(incol, errors='coerce')    # Convert to numeric, outliers to NaN
    if (dfs.isna().sum()) > 0 :                    # If outliers present, convert known items
        dfs = getnum (incol, '+', fromend=False)
        dfs = getnum (dfs, '-')
        dfs = pd.to_numeric(getnum (dfs, '<'), errors='coerce')
    return dfs
     
#Secondary function for clean up, split string based on 

def getnum (incol, str, fromend=True) :
    
    dfs = incol.str.split(str, n=2, expand=True)    # Split object on known delimeter
    
    if len(dfs.columns) == 2 :                      # If delimeter present
        if fromend :                                # Take numeric from before/after delimeter
            mask = ~(dfs[1].isnull())
            dfs.loc[mask,0] = dfs.loc[mask,1]
        return dfs[0]                               # Return extracted numeric
    
    else :
        return incol                                # No delimeter present, return input


In [ ]:
#Instantiate empty DataFrame for Opiod data set
opcols   = ['Unnamed: 0', 'STATEFP', 'COUNTYFP', 'YEAR', 'INDICATOR', 'VALUE', 
            'COUNTY', 'STATE', 'STATEABBREVIATION', 'Numeric']
opdf = pd.DataFrame(columns=opcols)

#Initialize reader for input file, specify input "chunk" size
file = 'data/countydata.csv'
rdr  = pd.read_csv(file, chunksize=50000)

In [ ]:
# Read input one chunk at a time, convert know value outliers to numeric
for chunk in rdr :
    chunk['Numeric'] = clean_nan(chunk['VALUE'])
    opdf = pd.concat([opdf, chunk])
    print(f'Processing chunk {chunk.index[0]} - {chunk.index[-1]}')
    
print(f'Processing complete, {opdf.shape[0]} records')

In [ ]:
#Still quite a few NaN in the input data
pd.to_numeric(opdf['VALUE'], errors='coerce').isna().sum(), opdf['Numeric'].isna().sum()

In [ ]:
typefile = 'data/indicator_types.csv'
typedf = pd.read_csv(typefile)
typedf.head()

In [ ]:
opdf = opdf.merge(typedf, on='INDICATOR', how='left')
opdf.head()

In [ ]:
pop = opdf[opdf['INDICATOR']=='Population'].groupby('STATEABBREVIATION')['Numeric'].sum()
deaths = opdf[opdf['INDICATOR']=='drugdeaths'].groupby('STATEABBREVIATION')['Numeric'].sum()

In [ ]:
pop1 = pop/10000

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(pop1.index, pop1.values)
plt.plot(deaths.index, deaths.values)